In [182]:
import cv2
import numpy as np
from pathlib import Path

In [183]:
PHOTO_PATH = '6.-Original-Damaged-Image-Woman.jpg'
RESULT_FOLDER = "results/"

ALPHA_VALUE = 1.4
SEGMENTATION_MIN = 250
INPAINT_RADIUS = 30
INPAINT_FLAG = cv2.INPAINT_TELEA

In [184]:
img = cv2.imread(PHOTO_PATH, cv2.IMREAD_GRAYSCALE)
stretched_image = cv2.normalize(img, None, 0, 230, cv2.NORM_MINMAX)
scaled_image = cv2.multiply(stretched_image,ALPHA_VALUE)

cv2.imwrite(filename=str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", f"_scale.jpg"))), 
            img=scaled_image)

__, damage_mask = cv2.threshold(scaled_image, SEGMENTATION_MIN, 255, cv2.THRESH_BINARY)
damage_mask = cv2.dilate(damage_mask, np.ones((5,5)))

cv2.imwrite(filename=str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", f"_d_mask.jpg"))), 
            img=damage_mask)

canny_edges = cv2.Canny(scaled_image, threshold1=254, threshold2=255)
canny_edges = cv2.dilate(canny_edges, np.ones((4,4)))

cv2.imwrite(filename=str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", f"_d_canny.jpg"))), 
            img=canny_edges)

damage_mask_sum = cv2.bitwise_or(canny_edges, damage_mask)

cv2.imwrite(filename=str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", f"_d_mask_sum.jpg"))), 
            img=damage_mask_sum)

img = cv2.imread(PHOTO_PATH)

filling_result = cv2.inpaint(img, damage_mask_sum, inpaintRadius=INPAINT_RADIUS, flags=INPAINT_FLAG)
cv2.imwrite(filename=str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", "_filled.jpg"))), 
            img=filling_result)

blur_result = cv2.GaussianBlur(filling_result, (3,3), 0)
sharpened = cv2.addWeighted(src1=blur_result, alpha=ALPHA_VALUE, src2=filling_result, beta=-0.5, gamma=0.2)

cv2.imwrite(filename = str(Path().joinpath(RESULT_FOLDER, PHOTO_PATH.replace(".jpg", "_final.jpg"))), 
            img=sharpened)

True